In [4]:
# importing modules
import arcpy
from arcpy import env
from sys import argv
from arcpy.sa import *

# adding timestamp for addMessage() replacement
def timestamp(message): # This is meant to replace print() or arcpy.AddMessage() in the script when I want to add messages.  It timestamps the messages so I can monitor performance of different stages in script.
    now = datetime.datetime.now()
    print(now.strftime("%H:%M:%S") + " - " + message)

try:
    if arcpy.CheckExtension("Spatial") == "Available":
        arcpy.CheckOutExtension("Spatial")
        print ("Checked out \"Spatial\" Extension")
    else:
        raise LicenseError
except LicenseError:
    print("Spatial Analyst license is unavailable")

timestamp("Modules loaded. Starting geoprocessing")

# adding speaking command so when each process completes it speaks
def speak(text):
    from IPython.display import Javascript as js, clear_output
    # Escape single quotes
    text = text.replace("'", r"\'")
    display(js(f'''
    if(window.speechSynthesis) {{
        var synth = window.speechSynthesis;
        synth.speak(new window.SpeechSynthesisUtterance('{text}'));
    }}
    '''))
    # Clear the JS so that the notebook doesn't speak again when reopened/refreshed
    clear_output(False)

Checked out "Spatial" Extension
23:34:27 - Modules loaded. Starting geoprocessing


In [8]:
'''
inputs
country, variable names ara meant to defne resulting perchange raster names in proper manner. This is helpful if you are working on multiple rasters
----------------------------------------------------------------
countrries list
BLZ, CRI, DOM, GTM, HND, HTI, MEX, NIC, PAN, SLV
variables list
NDVI, pdsi, Precipitation, Pollution, NTL, AirTemp
Note:  Agri, & (LULC) will not work with this script
----------------------------------------------------------------
'''
#inputs:
countries = ['BLZ', 'CRI', 'DOM', 'HND', 'HTI', 'MEX', 'NIC', 'PAN', 'SLV']
variable = 'AirTemp'

for c in countries:
    country = c
    pre_year = '2000'
    post_year = '2020'
    r_ext = '.tif'
    #input loc
    pre_raster_loc = "D:\\Work\\Fiverr\\Fieverr_IncomePoverty\\change_detection\\pre\\"
    post_raster_loc = "D:\\Work\\Fiverr\\Fieverr_IncomePoverty\\change_detection\\post\\"

    # Export Output
    export_loc = "D:\\Work\\Fiverr\\Fieverr_IncomePoverty\\change_detection\\final_export\\"
    temp_loc = "in_memory\\"
    # -----------------------------------------------------------
    pre_r = country + '_' + variable + '_' + pre_year + r_ext
    post_r = country + '_' + variable + '_' + post_year + r_ext

    pre_raster = pre_raster_loc + pre_r
    post_raster = post_raster_loc + post_r
    print('c name = ', c)
    print('Pre Raster loc: ', pre_raster)
    print('Post Raster Loc: ', post_raster)
    arcpy.env.overwriteOutput = True
    timestamp('Input output initialized successfully...')

    """
    PRE RASTER Normalization
    """
    pre_stat_raster = arcpy.management.CalculateStatistics(in_raster_dataset=pre_raster)
    pre_stat_raster = arcpy.Raster(pre_stat_raster)
    pre_normalized_name = pre_r[0:8] + '_nor_pre'
    pre_normalized_loc = temp_loc + pre_normalized_name + '.tif'
    # Post Normalization Analysis
    pre_normalized = (pre_stat_raster - pre_stat_raster.minimum) / (pre_stat_raster.maximum - pre_stat_raster.minimum)
    pre_normalized.save(pre_normalized_loc)

    """
    POST Raster Normalization
    """
    post_stat_raster = arcpy.management.CalculateStatistics(in_raster_dataset=post_raster)
    post_stat_raster = arcpy.Raster(post_stat_raster)
    post_normalized_name = post_r[0:8] + '_nor_post'
    post_normalized_loc = temp_loc + post_normalized_name + r_ext
    # Post Normalization Analysis
    post_normalized = (post_stat_raster - post_stat_raster.minimum) / (post_stat_raster.maximum - post_stat_raster.minimum)
    post_normalized.save(post_normalized_loc)
    timestamp('Pre and post normalization done...')

    """
    Percentage Change Analysis
    """
    perchange_name = post_r[0:8] + '_cd' + r_ext
    perchange_loc = export_loc + perchange_name
    perchange = ((post_normalized -  pre_normalized ) / ( pre_normalized )) * 100
    
    temp_perchange_loc = temp_loc + 'temp_' + perchange_name
    perchange.save(temp_perchange_loc)
    timestamp('percent change done...')

    """
    Removing values greater then 1000 (converting them to 0)

    """
    arcpy.CheckOutExtension("Spatial")
    inRaster = Raster(temp_perchange_loc)
    modified_perchange = Con(inRaster >= 1000, 0, inRaster)
    modified_perchange.save(perchange_loc)

    timestamp('Removing >1000 values done...')
    timestamp('---------COMPLETED - ({})---------'.format(perchange_name))


speak('-----This process is COMPLETED Please Run another code ')
    #------------------- Script END --------------------------